In [1]:
def inAllOthers(phrase, collect_commentary):
  not_to_flag = []
  for i in collect_commentary:
    found = False
    for j in i:
      if (phrase in j):
        found = True
        not_to_flag.append(j)
    if found == False:
      return False, not_to_flag
  return True, not_to_flag

In [2]:
def getSubstrateIndices(lit_indices, substrate_list):
  substrate_by_index_number = {}
  for i in lit_indices:
    sub = substrate_list[i]
    if sub in substrate_by_index_number:
      substrate_by_index_number[sub].append(i)
    else:
      substrate_by_index_number[sub] = [i]
  return substrate_by_index_number

In [3]:
def notAlreadyFound(phrase, similar, dissimilar):
  for i in similar:
    if phrase == i:
      return False 
  for i in dissimilar:
    if phrase == i:
      return False 
  return True

In [4]:
def inNotToFlag(not_to_flag, phrase):
  for i in not_to_flag:
    if i == phrase:
      return True 
  return False

In [5]:
def getSimilarCommentary(cluster, commentary):
  collect_commentary = []
  not_to_flag = []
  for index in cluster:
    if (commentary[index] is not None):
      comm_arr = commentary[index].split(', ')
      collect_commentary.append(comm_arr)
  similar = []
  different = []
  for arr in collect_commentary:
    for phrase in arr:
      boolean, ar = inAllOthers(phrase, collect_commentary)
      if (boolean and notAlreadyFound(phrase, similar, different)):
        similar.append(phrase)
        for a in ar:
          not_to_flag.append(a)
      if (not boolean and notAlreadyFound(phrase, similar, different) and not inNotToFlag(not_to_flag, phrase)):
        different.append(phrase)
  return similar, different

In [6]:
def getKMValueRange(cluster, kmValue):
  kmValues = []
  for i in cluster:
    kmValues.append(kmValue[i])
  kmValues.sort()
  return kmValues[0], kmValues[len(kmValues) - 1]

In [13]:
def createTableColumnsSequence(legitimate_groups, organism, literature, commentary, kmValue, substrate):
  count = 1
  #confidence_scores = []
  #similar_commentary = []
  #different_commentary = []
  for seq in legitimate_groups.keys():
    print("Sequence Cluster " + str(count) + ": " + seq)
    index = legitimate_groups[seq][0]
    print("     Literature: " + str(literature[index]))
    min, max = getKMValueRange(legitimate_groups[seq], kmValue)
    print("     KM Value Range: " + str(min) + " - " + str(max))
    similar, dissimilar = getSimilarCommentary(legitimate_groups[seq], commentary)
    #similar_commentary.append(similar)
    #different_commentary.append(dissimilar)
    print("     Similar Commentary: " + str(makeString(similar)))
    #print("     Dissimilar Commentary: " + str(makeString(dissimilar)))
    if (len(similar) + len(dissimilar) == 0):
      confidence_score = "NaN"
    else:
      confidence_score = float(len(similar))/float(len(dissimilar) + len(similar))
    #confidence_scores.append(confidence_score)
    #print("     Confidence Score: " + str(confidence_score))
    print("     Elements in this Cluster: ")
    for i in legitimate_groups_two[seq]:
      print("         " + organism[i] + " | " + kmValue[i] + " | " + commentary[i] + " | " + substrate[i])
    print()
    count = count + 1
  #return confidence_score, similar_commentary, different_commentary

In [8]:
def makeString(arr):
  phrase = ""
  for word in range(0, len(arr)):
    phrase += arr[word] + ", "
  return phrase

In [9]:
def sameCommentary(seq_group, commentary):
  to_delete = []
  for i in seq_group:
    similarity_scores = []
    for j in seq_group: 
      if i != j:
        if (commentary[i] is not None and commentary[j] is not None):
          similarity_scores.append(difflib.SequenceMatcher(None, commentary[i], commentary[j]).ratio())
        else:
          similarity_scores.append(0.0)
    count_one = 0
    for k in similarity_scores:
      if k == 1.0:
        count_one = 1 + count_one 
    if count_one < (len(similarity_scores) - count_one):
      to_delete.append(i)
  return to_delete

In [10]:
cofactors = ["H2O", "Water",
"ATP", "Adenosine 5’-triphosphate",
"NAD+", "NAD", "Nicotinamide adenine dinucleotide", "DPN", "Diphosphopyridine nucleotide", "Nadide", "beta-NAD+",
"NADH", "DPNH", "Reduced nicotinamide adenine dinucleotide",
"NADPH", "TPNH", "Reduced nicotinamide adenine dinucleotide phosphate",
"NADP+", "NADP", "Nicotinamide adenine dinucleotide phosphate", "beta-Nicotinamide adenine dinucleotide phosphate", "TPN", "Triphosphopyridine nucleotide", "beta-NADP+",
"Oxygen", "O2",
"ADP", "Adenosine 5'-diphosphate",
"CoA", "Coenzyme A", "CoA-SH",
"UDP", "Uridine 5'-diphosphate",
"FAD", "Flavin adenine dinucleotide"
"AMP", "Adenosine 5'-monophosphate", "Adenylic acid", "Adenylate", "5'-AMP", "5'-Adenylic acid", "5'-Adenosine monophosphate", "Adenosine 5'-phosphate",
"Acetyl-CoA", "Acetyl coenzyme A",
"GDP", "Guanosine 5'-diphosphate", "Guanosine diphosphate",
"GTP", "Guanosine 5'-triphosphate",
"Glutathione; 5-L-Glutamyl-L-cysteinylglycine", "N-(N-gamma-L-Glutamyl-L-cysteinyl)glycine", "gamma-L-Glutamyl-L-cysteinyl-glycine", "GSH", "Reduced glutathione",
"3'-Phosphoadenylyl sulfate", "3'-Phosphoadenosine 5'-phosphosulfate", "3'-Phospho-5'-adenylyl sulfate", "PAPS",
"Adenosine 3',5'-bisphosphate", "PAP", "3'-Phosphoadenylate", "Phosphoadenosine phosphate",
"CMP", "Cytidine-5'-monophosphate", "Cytidylic acid",
"CTP", "Cytidine 5'-triphosphate", "Cytidine triphosphate",
"Thiamin diphosphate", "Thiamine diphosphate", "Thiamin pyrophosphate", "TPP", "ThPP",
"UTP", "Uridine 5'-triphosphate", "Uridine triphosphate",
"ITP", "Inosine 5'-triphosphate", "Inosine triphosphate", "Inosine tripolyphosphate",
"Sulfur", "S", "Elemental sulfur",
"IDP", "Inosine 5'-diphosphate", "Inosine diphosphate",
"UMP", "Uridylic acid", "Uridine monophosphate", "Uridine 5'-monophosphate", "5'Uridylic acid",
"CDP", "Cytidine 5'-diphosphate", "Cytidine diphosphate",
"IMP", "Inosinic acid", "Inosine monophosphate", "Inosine 5'-monophosphate", "Inosine 5'-phosphate", "5'-Inosinate", "5'-Inosinic acid", "5'-Inosine monophosphate", "5'-IMP",
"dATP", "2'-Deoxyadenosine 5'-triphosphate", "Deoxyadenosine 5'-triphosphate", "Deoxyadenosine triphosphate",
"GMP", "Guanosine 5'-phosphate", "Guanosine monophosphate", "Guanosine 5'-monophosphate", "Guanylic acid",
"UDP-glucuronate," "UDPglucuronate", "UDP-D-glucuronate", "UDP-alpha-D-glucuronate",
"dADP", "2'-Deoxyadenosine 5'-diphosphate",
"dCMP", "Deoxycytidylic acid", "Deoxycytidine monophosphate", "Deoxycytidylate", "2'-Deoxycytidine 5'-monophosphate",
"dGTP", "2'-Deoxyguanosine 5'-triphosphate", "Deoxyguanosine 5'-triphosphate", "Deoxyguanosine triphosphate",
"dAMP", "2'-Deoxyadenosine 5'-phosphate", "2'-Deoxyadenosine 5'-monophosphate", "Deoxyadenylic acid", "Deoxyadenosine monophosphate",
"dGDP", "2'-Deoxyguanosine 5'-diphosphate",
"dGMP", "2'-Deoxyguanosine 5'-monophosphate", "2'-Deoxyguanosine 5'-phosphate", "Deoxyguanylic acid", "Deoxyguanosine monophosphate",
"dTDP", "Deoxythymidine 5'-diphosphate",
"dTMP", "Thymidine 5'-phosphate", "Deoxythymidine 5'-phosphate", "Thymidylic acid", "5'-Thymidylic acid", "Thymidine monophosphate", "Deoxythymidylic acid", "Thymidylate",
"dUMP", "Deoxyuridylic acid", "Deoxyuridine monophosphate", "Deoxyuridine 5'-phosphate", "2'-Deoxyuridine 5'-phosphate",
"dCTP", "Deoxycytidine 5'-triphosphate", "Deoxycytidine triphosphate", "2'-Deoxycytidine 5'-triphosphate",
"dTTP", "Deoxythymidine triphosphate", "Deoxythymidine 5'-triphosphate", "TTP",
"dUTP", "2'-Deoxyuridine 5'-triphosphate",
"3',5'-Cyclic AMP", "Cyclic adenylic acid; Cyclic AMP", "Adenosine 3',5'-phosphate", "Adenosine 3',5'-cyclic phosphate", "cAMP",
"Xanthosine 5'-phosphate", "Xanthylic acid", "XMP", "(9-D-Ribosylxanthine)-5'-phosphate",
"XTP", "Xanthosine 5'-triphosphate",
"dCDP", "2'-Deoxycytidine diphosphate", "2'-Deoxycytidine 5'-diphosphate",
"3',5'-Cyclic GMP", "Guanosine 3',5'-cyclic monophosphate", "Guanosine 3',5'-cyclic phosphate", "Cyclic GMP", "cGMP",
"dIDP", "2'-Deoxyinosine-5'-diphosphate", "2'-Deoxyinosine 5'-diphosphate",
"dITP", "2'-Deoxyinosine-5'-triphosphate", "2'-Deoxyinosine 5'-triphosphate",
"dUDP", "2'-Deoxyuridine 5'-diphosphate",
"FADH2",
"3'-AMP", "3'-Adenylic acid", "3'-Adenosine monophosphate", "Adenosine-3'-monophosphate", "Adenosine 3'-phosphate", "AMP 3'-phosphate",
"3'-UMP", "Uridine 3'-monophosphate", "Uridine 3'-phosphate",
"2',3'-Cyclic AMP",
"2',3'-Cyclic CMP",
"2',3'-Cyclic UMP",
"1-(5-Phospho-D-ribosyl)-ATP", "Phosphoribosyl-ATP", "N1-(5-Phospho-D-ribosyl)-ATP", "1-(5-Phosphoribosyl)-ATP", "1-(5-Phospho-beta-D-ribosyl)-ATP",
"Phosphoribosyl-AMP", "N1-(5-Phospho-D-ribosyl)-AMP", "1-(5-Phosphoribosyl)-AMP", "1-(5-Phospho-beta-D-ribosyl)-AMP",
"ADPribose 2'-phosphate",
"N6-(1,2-Dicarboxyethyl)-AMP", "N6-(1,2-Dicarboxyethyl)AMP", "Adenylosuccinate", "Adenylosuccinic acid",
"3'-CMP", "Cytidine 3'-phosphate",
"Hydrogen peroxide", "H2O2", "Oxydol",
"Superoxide; O2.-", "Superoxide anion", "O2-",
"HCO3", "Hydrogencarbonate", "Bicarbonate", "Hydrogen carbonate", "Acid carbonate",
"Formate", "Methanoic acid; Formic acid",
"H+", "Hydron"]

In [14]:
!pip install zeep
from zeep import Client
import hashlib
import difflib
ec_nums = ['5.1.1.1', '5.1.1.10', '5.1.1.11', '5.1.1.12', '5.1.1.13', '5.1.1.14', '5.1.1.15', '5.1.1.16', '5.1.1.17', '5.1.1.18', '5.1.1.19', '5.1.1.2', '5.1.1.20', '5.1.1.21', '5.1.1.22', '5.1.1.23', '5.1.1.24', '5.1.1.3', '5.1.1.4', '5.1.1.5', '5.1.1.6', '5.1.1.7', '5.1.1.8', '5.1.1.9']
wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256("iris12margo".encode("utf-8")).hexdigest()
client = Client(wsdl)
for num in ec_nums:
  ec_number = num
  parameters = ("elizabeth.cucuzzella@tufts.edu", password, "ecNumber*" + ec_number, "organism*", "kmValue*", "kmValueMaximum*", "substrate*", "commentary*", "ligandStructureId*", "literature*")
  try:
    resultString = client.service.getKmValue(*parameters)
  except (SyntaxError, RuntimeError, ValueError, NameError):
    resultString = ""
  substrate, organism, kmValue, commentary, literature = [], [], [], [], []
  for element in resultString:
    substrate.append(element['substrate'])
    organism.append(element['organism'])
    kmValue.append(element['kmValue'])
    commentary.append(element['commentary'])
    literature.append((element['literature'])[0])
  lit_by_index_number = {}
  index = 0
  for n in literature:
    if (n in lit_by_index_number):
      lit_by_index_number[n].append(index)
    else:
      lit_by_index_number[n] = [index]
    index = index + 1
  legitimate_groups_two = {}
  for literature_number in lit_by_index_number.keys():
    seq_by_index_number = getSubstrateIndices(lit_by_index_number[literature_number], organism)
    for seq in seq_by_index_number.keys():
      if (len(seq_by_index_number[seq]) > 1 and seq not in legitimate_groups_two):
        legitimate_groups_two[seq] = seq_by_index_number[seq]
  for seq in legitimate_groups_two.keys():
    for j in legitimate_groups_two[seq]:
      for element in cofactors:
        if (substrate[j] == element):
          legitimate_groups_two[seq].remove(j)
  count = 0
  to_remove = []
  for seq in legitimate_groups_two.keys():
    to_remove.append(sameCommentary(legitimate_groups_two[seq], commentary))
    count = count + 1
  count = 0
  for seq in legitimate_groups_two.keys():
    for index in to_remove[count]:
      if index in legitimate_groups_two[seq]:
        legitimate_groups_two[seq].remove(index)
    count = count + 1
  seq_to_remove = []
  for seq in legitimate_groups_two.keys():
    if len(legitimate_groups_two[seq]) == 0:
      seq_to_remove.append(seq)
  for i in seq_to_remove:
    del legitimate_groups_two[i]
  createTableColumnsSequence(legitimate_groups_two, organism, literature, commentary, kmValue, substrate)

Sequence Cluster 1: Procambarus clarkii
     Literature: 650876
     KM Value Range: 171 - 73.5
     Similar Commentary: 30&deg;C, pH 8.5, 
     Elements in this Cluster: 
         Procambarus clarkii | 73.5 | 30&deg;C, pH 8.5 | D-Ala
         Procambarus clarkii | 171 | 30&deg;C, pH 8.5 | L-Ala

Sequence Cluster 2: Staphylococcus aureus
     Literature: 748310
     KM Value Range: 0.46 - 0.97
     Similar Commentary: pH and temperature not specified in the publication, 
     Elements in this Cluster: 
         Staphylococcus aureus | 0.46 | pH and temperature not specified in the publication | D-alanine
         Staphylococcus aureus | 0.97 | pH and temperature not specified in the publication | L-alanine

Sequence Cluster 3: Enterococcus faecalis
     Literature: 748310
     KM Value Range: 0.48 - 6.8
     Similar Commentary: pH and temperature not specified in the publication, 
     Elements in this Cluster: 
         Enterococcus faecalis | 0.48 | pH and temperature not specified i